<a href="https://colab.research.google.com/github/Keerthu8999/GoogleColab/blob/main/LatestCrimeOnlyGit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.4 MB/s

In [ ]:
import logging

import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
data = pd.read_csv("/content/sample_data/CrimeTrain.csv",encoding="latin1", header=None )
data = data.dropna()
data.columns = ["sentence_id", "words", "labels"]
data['labels'] = data['labels'].str.upper()
data.head(30)

In [ ]:
unique_labels = data['labels'].unique().tolist()
print(unique_labels)


In [ ]:
eval_data = pd.read_csv("/content/sample_data/Crimetest.csv",encoding="latin1", header=None )
eval_data = eval_data.dropna()
eval_data.columns = ["sentence_id", "words", "labels"]
eval_data['labels'] = eval_data['labels'].str.upper()
eval_data.head(30)

In [ ]:
# Configure the model
model_args = NERArgs(overwrite_output_dir=True)
model_args.num_train_epochs = 1
model_args.learning_rate = 1e-4
model_args.overwrite_output_dir =True
model_args.train_batch_size = 32
model_args.eval_batch_size = 32

'''model = NERModel(
    "roberta", "roberta-base", args=model_args, use_cuda=False

)
'''
model = NERModel('bert', 'bert-base-cased',labels=unique_labels,args =model_args, use_cuda = False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(data, eval_data=eval_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, preds_list = model.eval_model(eval_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model.predict(["Hermione was the best in her class"])
print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Hermione': 'O'}, {'was': 'O'}, {'the': 'O'}, {'best': 'O'}, {'in': 'O'}, {'her': 'O'}, {'class': 'O'}]]


In [1]:
from huggingface_hub import notebook_login
notebook_login()


In [2]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
model.push_to_hub(
    repo_id="keer-crime-model",
    organization="Keertss",
    commit_message="Add model",
    use_temp_dir=True,
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "Keertss/keer-crime-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)